In [ ]:

!pip install imageio[ffmpeg] opencv-python tensorflow


import cv2
import imageio
import numpy as np
import tensorflow as tf
from google.colab import files
import matplotlib.pyplot as plt

# Function to perform dark channel prior dehazing on a single frame
def dehaze_frame(frame):
    # Convert BGR image to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Normalize pixel values to [0, 1]
    frame = frame / 255.0

    # Calculate dark channel
    dark_channel = np.min(frame, axis=2)

    # Estimate atmospheric light
    top_percent = 0.1
    pixel_count = int(np.prod(frame.shape[:2]) * top_percent)
    flat_dark_channel = dark_channel.flatten()
    indices = np.argpartition(flat_dark_channel, -pixel_count)[-pixel_count:]
    atmospheric_light = np.max(frame.reshape(-1, 3)[indices], axis=0)

    # Estimate transmission
    omega = 0.95
    transmission = 1 - omega * dark_channel / np.expand_dims(atmospheric_light, axis=(1, 2))
    # Clip transmission to avoid artifacts
    transmission = np.clip(transmission, 0.1, 1.0)

    # Dehaze the frame
    dehazed_frame = np.empty_like(frame)
    for i in range(3):
        atmospheric_light = atmospheric_light.reshape((1, 1, 3))
        dehazed_frame[:, :, i] = (frame[:, :, i] - np.expand_dims(atmospheric_light[i], axis=(0, 1))) / transmission + atmospheric_light[i]
    # Clip pixel values to [0, 1]
    dehazed_frame = np.clip(dehazed_frame, 0, 1)

    return (dehazed_frame * 255).astype(np.uint8)

# Function to dehaze a video
def dehaze_video(input_path, output_path):
    video_reader = imageio.get_reader(input_path)
    fps = video_reader.get_meta_data()['fps']
    video_writer = imageio.get_writer(output_path, fps=fps)

    for frame in video_reader:
        dehazed_frame = dehaze_frame(frame)
        video_writer.append_data(dehazed_frame)

    video_writer.close()


uploaded = files.upload()

# Use the uploaded video as input
input_video_path = list(uploaded.keys())[0]
output_video_path = "dehazed_video.mp4"

# Perform dehazing on the video
dehaze_video(input_video_path, output_video_path)

# Display the original and dehazed videos
original_video = cv2.VideoCapture(input_video_path)
dehazed_video = cv2.VideoCapture(output_video_path)

# Display original video
print("Original Video:")
while True:
    ret, frame = original_video.read()
    if not ret:
        break
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.show()

# Display dehazed video
print("Dehazed Video:")
while True:
    ret, frame = dehazed_video.read()
    if not ret:
        break
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.show()

# Release video capture objects
original_video.release()
dehazed_video.release()


Saving 400.mp4 to 400.mp4


ValueError: operands could not be broadcast together with shapes (720,1280) (1,1,1,3) 